In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\ashwi\\OneDrive\\Desktop\\devrev07_1\\under-water-animal-classification'

MLFLOW_TRACKING_URI=https://dagshub.com/ASHWIN492/under-water-animal-classification.mlflow \
MLFLOW_TRACKING_USERNAME=ASHWIN492 \
MLFLOW_TRACKING_PASSWORD=1055eec714afcdcbea6709267f829d12f5939166 \
python script.py

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ASHWIN492/under-water-animal-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ASHWIN492"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1055eec714afcdcbea6709267f829d12f5939166"

set MLFLOW_TRACKING_URI= https://dagshub.com/ASHWIN492/under-water-animal-classification.mlflow

set MLFLOW_TRACKING_USERNAME=ASHWIN492

set MLFLOW_TRACKING_PASSWORD=1055eec714afcdcbea6709267f829d12f5939166

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories 

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion",
            mlflow_uri="https://dagshub.com/ASHWIN492/under-water-animal-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [15]:
import mlflow
from urllib.parse import urlparse

from pathlib import Path
import tensorflow as tf

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.print_score()

    def print_score(self):
        loss, accuracy = self.score[0], self.score[1]
        print(f"Validation Loss: {loss:.4f}")
        print(f"Validation Accuracy: {accuracy * 100:.2f}%")

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="InceptionV3Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [16]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-01-14 03:32:18,340: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-14 03:32:18,358: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 03:32:18,364: INFO: common: created directory at: artifacts]
Found 4104 images belonging to 23 classes.
257/257 [==============================] - 281s 1s/step - loss: 0.6780 - accuracy: 0.8190
Validation Loss: 0.6780
Validation Accuracy: 81.90%


2024/01/14 03:37:13 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-14 03:37:38,293: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 95). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ashwi\AppData\Local\Temp\tmpsie85bxd\model\data\model\assets
[2024-01-14 03:37:45,060: INFO: builder_impl: Assets written to: C:\Users\ashwi\AppData\Local\Temp\tmpsie85bxd\model\data\model\assets]


c:\Users\ashwi\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'InceptionV3Model'.
2024/01/14 03:39:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: InceptionV3Model, version 1
Created version '1' of model 'InceptionV3Model'.
